In [1]:
import os
CORPUS_PATH = os.path.join('data', 'austen-bronte-split')
filenames = sorted([os.path.join(CORPUS_PATH, fn) for fn in os.listdir(CORPUS_PATH)])
print(len(filenames))

813


In [2]:
import os
CORPUS_PATH = os.path.join('data', 'austen-bronte-split')
filenames = sorted([os.path.join(CORPUS_PATH, fn) for fn in os.listdir(CORPUS_PATH)])
print(len(filenames))

import numpy as np
import sklearn.feature_extraction.text as text
vectorizer = text.CountVectorizer(input='filename', stop_words='english', min_df=1)
dtm = vectorizer.fit_transform(filenames).toarray()
vocab = np.array(vectorizer.get_feature_names())
print(vocab)

813


[u'000' u'10' u'10th' ..., u'zornes' u'zschokke' u'zurueck']


In [3]:
import os
CORPUS_PATH = os.path.join('data', 'austen-bronte-split')
filenames = sorted([os.path.join(CORPUS_PATH, fn) for fn in os.listdir(CORPUS_PATH)])
print(len(filenames))

import numpy as np
import sklearn.feature_extraction.text as text
vectorizer = text.CountVectorizer(input='filename', stop_words='english', min_df=1)
dtm = vectorizer.fit_transform(filenames).toarray()
vocab = np.array(vectorizer.get_feature_names())
print(vocab)

from sklearn import decomposition
num_topics = 20
num_top_words = 20
clf = decomposition.NMF(n_components=num_topics, random_state=1)
doctopic = clf.fit_transform(dtm)
topic_words = []
# components_ : array-like, shape (n_components, n_features) Factorization matrix, sometimes called 'dictionary'.
for topic in clf.components_:
    # Returns the indices that would sort an array.
    word_idx = np.argsort(topic)[::-1][0:num_top_words]
    topic_words.append([vocab[i] for i in word_idx])
doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)
print(topic_words)

813


[u'000' u'10' u'10th' ..., u'zornes' u'zschokke' u'zurueck']


[[u'said', u'think', u'know', u'come', u'did', u'replied', u'tell', u'cried', u'looked', u'answer', u'man', u'let', u'better', u'asked', u'yes', u'eyes', u'thought', u'mean', u'wish', u'door'], [u'mr', u'elton', u'knightley', u'man', u'think', u'did', u'dear', u'young', u'great', u'evening', u'time', u'quite', u'woodhouse', u'long', u'day', u'saw', u'business', u'told', u'know', u'wickham'], [u'marianne', u'elinor', u'willoughby', u'sister', u'jennings', u'mother', u'colonel', u'dashwood', u'time', u'brandon', u'mrs', u'moment', u'soon', u'heart', u'barton', u'spirits', u'mind', u'away', u'little', u'day'], [u'elizabeth', u'darcy', u'mr', u'bingley', u'bennet', u'sister', u'collins', u'wickham', u'lady', u'soon', u'lydia', u'catherine', u'family', u'time', u'miss', u'little', u'room', u'friend', u'mother', u'lizzy'], [u'did', u'know', u'time', u'heart', u'felt', u'told', u'thought', u'look', u'day', u'knew', u'believe', u'think', u'saw', u'night', u'feelings', u'came', u'paul', u'lette

In [4]:
novel_names = []
# generates the novel names according the file names (stripping the numbers) 
for fn in filenames:
    basename = os.path.basename(fn)
    name, ext = os.path.splitext(basename)
    name = name.rstrip('0123456789')
    novel_names.append(name)

# 
novel_names = np.asarray(novel_names)
doctopic_orig = doctopic.copy()
num_groups = len(set(novel_names))
doctopic_grouped = np.zeros((num_groups, num_topics))
for i, name in enumerate(sorted(set(novel_names))):
    doctopic_grouped[i, :] = np.mean(doctopic[novel_names == name, :], axis=0)
doctopic = doctopic_grouped

for t in range(len(topic_words)):
    print("Topic {}: {}".format(t, ' '.join(topic_words[t][:15])))

Topic 0: said think know come did replied tell cried looked answer man let better asked yes
Topic 1: mr elton knightley man think did dear young great evening time quite woodhouse long day
Topic 2: marianne elinor willoughby sister jennings mother colonel dashwood time brandon mrs moment soon heart barton
Topic 3: elizabeth darcy mr bingley bennet sister collins wickham lady soon lydia catherine family time miss
Topic 4: did know time heart felt told thought look day knew believe think saw night feelings
Topic 5: mrs did good room come lady day quite elton said time jennings thing weston came
Topic 6: jane fairfax love oh heart little think heard soon mother feel voice elton campbell god
Topic 7: little graham bretton papa like lucy polly dr child home paulina don eyes thought think
Topic 8: john st dr diana mary house like rivers home left sisters brother dashwood hannah asked
Topic 9: edward elinor lucy ferrars dashwood mother sister time say think sure brother soon thing make
Topic 

In [8]:
novels = sorted(set(novel_names))
for i in range(len(doctopic)):
    top_topics = np.argsort(doctopic[i,:])[::-1][0:3]
    top_topics_str = ' '.join(str(t) for t in top_topics)
    print("{}: {}".format(novels[i], top_topics_str))

Austen_Emma: 19 1 16
Austen_Pride: 3 1 18
Austen_Sense: 2 9 5
CBronte_Jane: 13 15 0
CBronte_Professor: 14 13 17
CBronte_Villette: 13 10 7
